In [1]:
#https://github.com/ventolab/CellphoneDB/tree/master/notebooks
##conda activate cpdb
#cellphonedb method statistical_analysis covid_CD16mono_platelet_meta.txt covid_CD16mono_platelet_count.txt --counts-data=gene_name#
#cellphonedb plot dot_plot --rows in/rows.txt --columns in/columns.txt

In [2]:
import pandas as pd
import scanpy as sc

# data after filtering and normalising
adata = sc.read('/home/jovyan/mounting_point/renal_covid_19_gex_myeloid_soupX_QC_orig.ident_annotated_levle23.h5ad')


In [3]:
raw= sc.read('/home/jovyan/data/covid_19_renal/renal_covid_19_gex_myeloid_soupX_QC.h5ad')

In [4]:
mdata = raw[raw.obs.index.isin(adata.obs.index)]

In [5]:
mdata

View of AnnData object with n_obs × n_vars = 126622 × 33559

In [6]:
adata

AnnData object with n_obs × n_vars = 126622 × 1215
    obs: 'annotation22_ES', 'centre', 'sample_id', 'sample_id_broad', 'sample_date', 'sample_date_yr', 'pool', 'pool_broad', 'haniffa_broad_predLabel', 'orig.ident', 'merged_souporcell_cluster', 'merged_souporcell_status', 'patient_id', 'case_control', 'WHO_severity', 'sex', 'calc_age', 'discharge_date', 'date_positive_swab', 'date_first_symptoms', 'admission_date', 'ethnicity', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden_0.8', '5_subcluster', '9_subcluster', 'annotation', 'annotation23_ES', 'annotation24_ES', 'annot3', 'annot2', 'annot1', 'leiden_0.5', 'annot4'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '5_subcluster_colors', '9_subcluster', '9_subcluster_colors', 'WHO_severity_colors', 'X_pca_harmony_orig.ident', 'annot1_colors', 'annot2_colors', 'annot3_colors', '

In [7]:
raw

AnnData object with n_obs × n_vars = 220907 × 33559

In [8]:
mdata.obs['annot4'] = adata.obs['annot4']
mdata.obs['centre'] = adata.obs['centre']
mdata.obs['case_control'] = adata.obs['case_control']
mdata.obs['WHO_severity'] = adata.obs['WHO_severity']
mdata.obs['patient_id'] = adata.obs['patient_id']
mdata.obs['sample_id'] = adata.obs['sample_id']

/tmp/ipykernel_11561/1601135058.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  mdata.obs['annot4'] = adata.obs['annot4']
/home/jovyan/my-conda-envs/Py_env2/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [9]:
mdata.obs['annot4'].unique()

['CD14mono_activated', 'CD16mono', 'pDC', 'CD14mono', 'Int.mono', ..., 'HSPC', 'DC3', 'DC1', 'DC3_IFN', 'ASDC']
Length: 17
Categories (17, object): ['ASDC', 'CD14mono', 'CD14mono_IFN', 'CD14mono_activated', ..., 'Int.mono', 'Platelet', 'RBC', 'pDC']

In [10]:
#sc.pp.subsample(adata, fraction=None, n_obs=100, random_state=0, copy=False)#subsample

In [11]:
mdata.obs['annot4'] = mdata.obs['annot4'].str.replace('CD14mono_activated','CD14mono')
mdata.obs['annot4'] = mdata.obs['annot4'].str.replace('CD14mono_anti_inflammatory','CD14mono_alt_act')

In [12]:
list(mdata.obs['annot4'].unique())

['CD14mono',
 'CD16mono',
 'pDC',
 'Int.mono',
 'CD16mono_C1',
 'CD16mono_IFN',
 'CD14mono_IFN',
 'RBC',
 'DC2',
 'Platelet',
 'CD14mono_alt_act',
 'HSPC',
 'DC3',
 'DC1',
 'DC3_IFN',
 'ASDC']

In [13]:
cell_interested=['CD14mono_alt_act','CD16mono','CD16mono_IFN',
                 'CD16mono_C1','CD14mono','CD14mono_IFN','Int.mono','Platelet']
sub_set=mdata[mdata.obs['annot4'].isin(cell_interested)]


In [14]:
list(sub_set.obs['annot4'].unique())

['CD14mono',
 'CD16mono',
 'Int.mono',
 'CD16mono_C1',
 'CD16mono_IFN',
 'CD14mono_IFN',
 'Platelet',
 'CD14mono_alt_act']

In [15]:
'C141' in sub_set.obs['patient_id'].unique()

True

In [16]:
'C139' in sub_set.obs['patient_id'].unique()

False

In [17]:
'C116' in sub_set.obs['patient_id'].unique()

False

In [19]:
# removing recover samples of C141
C141 = sub_set[sub_set.obs['patient_id']=='C141']

In [20]:
C141.obs['sample_id'].unique()

['C141_090321', 'C141_020221', 'C141_200121', 'C141_300121', 'C141_220121', 'C141_260121']
Categories (6, object): ['C141_020221', 'C141_090321', 'C141_200121', 'C141_220121', 'C141_260121', 'C141_300121']

In [21]:
C141_rec = C141[C141.obs['sample_id']=='C141_090321']

In [22]:
C141_rec.obs['case_control'].unique()

['RECOVERY']
Categories (1, object): ['RECOVERY']

In [23]:
sub_set

View of AnnData object with n_obs × n_vars = 104673 × 33559
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'patient_id', 'sample_id'

In [24]:
sub_set = sub_set[sub_set.obs['sample_id']!='C141_090321']

In [25]:
sub_set

View of AnnData object with n_obs × n_vars = 104347 × 33559
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'patient_id', 'sample_id'

# geosketch NR

In [26]:
sub_set.obs['annot4'].value_counts()

CD14mono            47175
CD16mono            21269
CD14mono_IFN        12965
CD16mono_IFN         6772
Int.mono             6126
CD16mono_C1          3522
Platelet             3522
CD14mono_alt_act     2996
Name: annot4, dtype: int64

In [27]:
adata_scaled = sub_set[:]

In [28]:
sc.pp.normalize_total(adata_scaled, target_sum=1e4)
sc.pp.log1p(adata_scaled)
sc.pp.scale(adata_scaled, max_value=10)

/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [29]:
from geosketch import gs
# subsample adata to 30k with geosketch
# use the scaled values from adata_filter to do the subsetting
X = adata_scaled.X
# Compute PCs.
from fbpca import pca
U, s, Vt = pca(X, k=100) # E.g., 100 PCs.
X_dimred = U[:, :100] * s[:100]
N = 20000 # Number of samples to obtain from the data set.


In [30]:
sketch_index = gs(X_dimred, N, replace=False)
#adata_filter_30k = adata_scaled[sketch_index]
#sc.pl.umap(adata_filter_30k, color = ['timeline'], ncols=2)
# now to subsample the actual adata and save it
adata_30k =  sub_set[sketch_index]
# save the file
#resultFile30k = '/home/jovyan/Tom/GSE134809_adata_30k.h5ad'
#adata_30k.write(resultFile30k)

In [31]:
#Create a subsample to save on re sampling - only 40% of dataset to run faster
#grouped = adata.obs.groupby('timeline')
#df = grouped.apply(lambda x: x.sample(frac=0.05))#0.4
#df = df.droplevel('timeline')
#keep = df.index
#adata = adata[adata.obs.index.isin(keep)]

In [32]:
sub_set= adata_30k

In [33]:
sub_set

View of AnnData object with n_obs × n_vars = 20000 × 33559
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'patient_id', 'sample_id'

# end

In [34]:
sc.pp.normalize_total(sub_set, target_sum=1e4)

/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [35]:
sc.pp.log1p(sub_set)

In [ ]:
gene_interested=[]

sub_set1=sub_set[:, gene_interested]

In [ ]:
sub_set

In [36]:
sub_set1=sub_set

In [37]:
sub_set1

AnnData object with n_obs × n_vars = 20000 × 33559
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'patient_id', 'sample_id'
    uns: 'log1p'

In [38]:
# remove vdj and light chain constant genes from highly variable genes, and also the viral reads
import re
for i in sub_set1.var.index:
    if re.search('^IG[HKL][VDJC]|VIRAL', i):
        sub_set1.var.at[i, 'highly_variable'] = False
#sc.pl.highly_variable_genes(sub_set1)

In [39]:
sub_set1 = sub_set1[:, sub_set1.var.highly_variable!=False]
sub_set1

View of AnnData object with n_obs × n_vars = 20000 × 33141
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'patient_id', 'sample_id'
    var: 'highly_variable'
    uns: 'log1p'

In [40]:
sub_set1.var

,highly_variable
MIR1302-2HG,NaN
FAM138A,NaN
OR4F5,NaN
AL627309.1,NaN
AL627309.3,NaN
...,...
AC233755.2,NaN
AC233755.1,NaN
AC240274.1,NaN
AC213203.1,NaN


In [41]:
df_expr_matrix = sub_set1.X.T ####? should be raw.X???

In [42]:
df_expr_matrix.shape

(33141, 20000)

In [43]:
df_expr_matrix = pd.DataFrame(df_expr_matrix.toarray())

In [44]:
df_expr_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_expr_matrix.columns = sub_set1.obs.index

In [46]:
df_expr_matrix.head()

,UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,UK-CIC10689738+UK-CIC10690026_AAACGGGCAAGTACCT-1,UK-CIC10689738+UK-CIC10690026_AAACGGGCATTGCGGC-1,UK-CIC10689738+UK-CIC10690026_AAAGATGGTAGCGCAA-1,UK-CIC10689738+UK-CIC10690026_AAAGCAAGTACTTCTT-1,UK-CIC10689738+UK-CIC10690026_AAAGTAGTCAGTTCGA-1,UK-CIC10689738+UK-CIC10690026_AACACGTTCGTAGGTT-1,UK-CIC10689738+UK-CIC10690026_AACCATGAGTCACGCC-1,UK-CIC10689738+UK-CIC10690026_AACCGCGGTTCAACCA-1,...,UK-CIC10690384+UK-CIC10690576_TGTTCCGGTGTTCGAT-1,UK-CIC10690384+UK-CIC10690576_TTAGTTCGTAAGTTCC-1,UK-CIC10690384+UK-CIC10690576_TTAGTTCGTTCACCTC-1,UK-CIC10690384+UK-CIC10690576_TTCTCAAAGATAGGAG-1,UK-CIC10690384+UK-CIC10690576_TTCTCCTCATGACGGA-1,UK-CIC10690384+UK-CIC10690576_TTCTCCTGTCTAGGTT-1,UK-CIC10690384+UK-CIC10690576_TTCTTAGTCAATAAGG-1,UK-CIC10690384+UK-CIC10690576_TTGACTTAGAAACGCC-1,UK-CIC10690384+UK-CIC10690576_TTGTAGGGTTACCGAT-1,UK-CIC10690384+UK-CIC10690576_TTTACTGTCTGCAAGT-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
sub_set1.var

,highly_variable
MIR1302-2HG,NaN
FAM138A,NaN
OR4F5,NaN
AL627309.1,NaN
AL627309.3,NaN
...,...
AC233755.2,NaN
AC233755.1,NaN
AC240274.1,NaN
AC213203.1,NaN


In [48]:
#sub_set1.var.set_index('gene_ids',inplace=True)


In [49]:
df_expr_matrix.set_index(sub_set1.var.index, inplace=True) 

In [50]:
df_expr_matrix.head()

,UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,UK-CIC10689738+UK-CIC10690026_AAACGGGCAAGTACCT-1,UK-CIC10689738+UK-CIC10690026_AAACGGGCATTGCGGC-1,UK-CIC10689738+UK-CIC10690026_AAAGATGGTAGCGCAA-1,UK-CIC10689738+UK-CIC10690026_AAAGCAAGTACTTCTT-1,UK-CIC10689738+UK-CIC10690026_AAAGTAGTCAGTTCGA-1,UK-CIC10689738+UK-CIC10690026_AACACGTTCGTAGGTT-1,UK-CIC10689738+UK-CIC10690026_AACCATGAGTCACGCC-1,UK-CIC10689738+UK-CIC10690026_AACCGCGGTTCAACCA-1,...,UK-CIC10690384+UK-CIC10690576_TGTTCCGGTGTTCGAT-1,UK-CIC10690384+UK-CIC10690576_TTAGTTCGTAAGTTCC-1,UK-CIC10690384+UK-CIC10690576_TTAGTTCGTTCACCTC-1,UK-CIC10690384+UK-CIC10690576_TTCTCAAAGATAGGAG-1,UK-CIC10690384+UK-CIC10690576_TTCTCCTCATGACGGA-1,UK-CIC10690384+UK-CIC10690576_TTCTCCTGTCTAGGTT-1,UK-CIC10690384+UK-CIC10690576_TTCTTAGTCAATAAGG-1,UK-CIC10690384+UK-CIC10690576_TTGACTTAGAAACGCC-1,UK-CIC10690384+UK-CIC10690576_TTGTAGGGTTACCGAT-1,UK-CIC10690384+UK-CIC10690576_TTTACTGTCTGCAAGT-1
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
df_expr_matrix.shape

(33141, 20000)

In [52]:
#df_expr_matrix=df_expr_matrix.loc[~(df_expr_matrix==0).all(axis=1)]

In [53]:
df_expr_matrix.shape

(33141, 20000)

In [56]:
sub_set.obs['case_control'].unique()

['NEGATIVE', 'POSITIVE', 'nan', 'RECOVERY']
Categories (4, object): ['NEGATIVE', 'POSITIVE', 'RECOVERY', 'nan']

In [57]:
df_expr_matrix.to_csv('/home/jovyan/mounting_point/cellphoneDB_output/allmono_platelet_bothwave_allcase_count.txt',sep='\t')

In [58]:
sub_set1.obs

,annot4,centre,case_control,WHO_severity,patient_id,sample_id
UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,CD14mono,NCL,NEGATIVE,nan,C103,C103_2204
UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,CD16mono,NCL,NEGATIVE,nan,C103,C103_2204
UK-CIC10689738+UK-CIC10690026_AAACGGGCAAGTACCT-1,CD16mono,NCL,NEGATIVE,nan,C103,C103_2204
UK-CIC10689738+UK-CIC10690026_AAACGGGCATTGCGGC-1,CD14mono,NCL,NEGATIVE,nan,C103,C103_2204
UK-CIC10689738+UK-CIC10690026_AAAGATGGTAGCGCAA-1,Int.mono,NCL,NEGATIVE,nan,C103,C103_2204
...,...,...,...,...,...,...
UK-CIC10690384+UK-CIC10690576_TTCTCCTGTCTAGGTT-1,Int.mono,Cambridge,POSITIVE,mild,C170,C170_020221
UK-CIC10690384+UK-CIC10690576_TTCTTAGTCAATAAGG-1,CD16mono,Cambridge,nan,nan,unresolved,nan
UK-CIC10690384+UK-CIC10690576_TTGACTTAGAAACGCC-1,Int.mono,Cambridge,POSITIVE,mild,C190,C190_010221
UK-CIC10690384+UK-CIC10690576_TTGTAGGGTTACCGAT-1,CD14mono,Cambridge,POSITIVE,mild,C170,C170_020221


In [59]:
sub_set1.obs.reset_index(inplace=True)

In [60]:
sub_set1.obs.head()

,index,annot4,centre,case_control,WHO_severity,patient_id,sample_id
0,UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,CD14mono,NCL,NEGATIVE,nan,C103,C103_2204
1,UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,CD16mono,NCL,NEGATIVE,nan,C103,C103_2204
2,UK-CIC10689738+UK-CIC10690026_AAACGGGCAAGTACCT-1,CD16mono,NCL,NEGATIVE,nan,C103,C103_2204
3,UK-CIC10689738+UK-CIC10690026_AAACGGGCATTGCGGC-1,CD14mono,NCL,NEGATIVE,nan,C103,C103_2204
4,UK-CIC10689738+UK-CIC10690026_AAAGATGGTAGCGCAA-1,Int.mono,NCL,NEGATIVE,nan,C103,C103_2204


In [61]:
df_meta = pd.DataFrame(data={'Cell': list(sub_set1.obs['index']), 
                             'cell_type': list(sub_set1.obs['annot4'])})

In [62]:
df_meta.head()

,Cell,cell_type
0,UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,CD14mono
1,UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,CD16mono
2,UK-CIC10689738+UK-CIC10690026_AAACGGGCAAGTACCT-1,CD16mono
3,UK-CIC10689738+UK-CIC10690026_AAACGGGCATTGCGGC-1,CD14mono
4,UK-CIC10689738+UK-CIC10690026_AAAGATGGTAGCGCAA-1,Int.mono


In [63]:
df_meta.set_index('Cell',inplace=True)
df_meta.to_csv('/home/jovyan/mounting_point/cellphoneDB_output/allmono_platelet_bothwave_allcase_meta.txt', sep='\t')

In [ ]:
# end